In [1]:
# Imports
from biocrnpyler import *
from genelet import *
import pylab as plt
import numpy as np
from bokeh.layouts import row
from bokeh.io import export_png

import warnings
import bokeh.io
import bokeh.plotting

from bioscrape.simulator import py_simulate_model
from bioscrape.types import Model

In [2]:
# Create CRN for bistable switch
Core1_off = Species ('Core1')
Core2_off = Species ('Core2')

Core1 = Genelet(Core1_off, transcript = "rR1", activator = "dA2", inhibitor = "rR2" )
Core2 = Genelet(Core2_off, transcript = "rR2", activator = "dA1", inhibitor = "rR1" )
M_BiSwitch = Mixture(name = "Switch_test", components = [Core1, Core2], parameter_file = "default_parameters.txt")
repr(M_BiSwitch)
CRN_BiSwitch = M_BiSwitch.compile_crn()

# Write SBML file
#CRN_BiSwitch.write_sbml_file('BiSwitch_CRN.xml')

print(CRN_BiSwitch.pretty_print())

Species (18) = {0. Core1_OFF, 1. complex[dna[dA2]:Core1_OFF], 2. rna[rR1], 3. dna[dA2], 4. rna[rR2], 5. protein[RNAP], 6. protein[RNAseH], 7. complex[rna[rR2]:dna[dA2]], 8. complex[complex[dna[dA2]:Core1_OFF]:protein[RNAP]], 9. complex[Core1_OFF:protein[RNAP]], 10. complex[complex[rna[rR2]:dna[dA2]]:protein[RNAseH]], 11. Core2_OFF, 12. complex[Core2_OFF:dna[dA1]], 13. dna[dA1], 14. complex[dna[dA1]:rna[rR1]], 15. complex[complex[Core2_OFF:dna[dA1]]:protein[RNAP]], 16. complex[Core2_OFF:protein[RNAP]], 17. complex[complex[dna[dA1]:rna[rR1]]:protein[RNAseH]]}
Reactions (18) = [
0. Core1_OFF + dna[dA2] --> complex[dna[dA2]:Core1_OFF]        
        massaction: k_f(Core1_OFF,dna[dA2])=0.00394*Core1_OFF*dna[dA2]
1. complex[dna[dA2]:Core1_OFF] + rna[rR2] --> Core1_OFF + complex[rna[rR2]:dna[dA2]]        
        massaction: k_f(complex[dna[dA2]:Core1_OFF],rna[rR2])=0.0696*complex[dna[dA2]:Core1_OFF]*rna[rR2]
2. dna[dA2] + rna[rR2] --> complex[rna[rR2]:dna[dA2]]        
        massaction: k

In [3]:
# Bioscrape simulation with one core ON and other core OFF

io = {"complex_Core1_ON": 0.1, "dna_dA2": 1, "rna_rR2": 0, "Core2_OFF": 0.1, "dna_dA1": 1, "rna_rR1": 0.5, "protein_RNAseH": 1,
      "protein_RNAP":15}
timepoints = np.linspace(0, 28800, 1000)
R = CRN_BiSwitch.simulate_with_bioscrape(timepoints, initial_condition_dict = io)
timepoints = timepoints/3600

# For label convenience
x = 'Time (hrs)'
y = 'Concentration (uM)'

# Core 1 concentration/molecules
bokeh.io.output_notebook()
a = bokeh.plotting.figure(plot_width=400, plot_height=300, x_axis_label = x, y_axis_label = y)
a.circle(timepoints, R["Core1_OFF"], legend_label = "OFF Core 1", color = "blue")
a.circle(timepoints, R["complex_Core1_ON"], legend_label = "ON Core 1", color = "red")
a.legend.click_policy="hide"


# Core 2 concentration/molecules
b = bokeh.plotting.figure(plot_width=400, plot_height=300,  x_axis_label = x, y_axis_label = y)
b.circle(timepoints, R["Core2_OFF"], legend_label = "OFF Core 2", color = "darkblue")
b.circle(timepoints, R["complex_Core2_ON"], legend_label = "ON Core 2" , color = "darkred")
b.legend.click_policy="hide"

# Core 1 concentration/molecules
bokeh.io.output_notebook()
g = bokeh.plotting.figure(plot_width=400, plot_height=300, x_axis_label = x, y_axis_label = y)
g.circle(timepoints, R["complex_Core2_ON_protein_RNAP"], legend_label = "ON Core 2", color = "red")
g.circle(timepoints, R["complex_Core1_ON_protein_RNAP"], legend_label = "ON Core 1", color = "blue")
g.legend.click_policy="hide"
g.legend.location = "center_right"

i = bokeh.plotting.figure(plot_width=400, plot_height=300, x_axis_label = x, y_axis_label = y)
i.circle(timepoints, R["rna_rR1"], legend_label = "rR1", color = "red")
i.circle(timepoints, R["dna_dA1"], legend_label = "dA1", color = "orange")
i.legend.click_policy="hide"
i.legend.location = "center_right"

j = bokeh.plotting.figure(plot_width=400, plot_height=300, x_axis_label = x, y_axis_label = y)
j.circle(timepoints, R["rna_rR2"], legend_label = "rR2", color = "blue")

bokeh.io.show(row(a, b, g, i, j))
warnings.filterwarnings("ignore")


Loading BokehJS ...

Loading BokehJS ...

In [4]:
from bioscrape.sbmlutil import import_sbml
P = import_sbml ('Biswitch_CRN.xml')

#Initials

# M = Model(species = [Core1,Core2], parameters = "default_parameters.txt", reactions = P, initial_condition_dict = io)

P.set_species({"complex_Core1_ON": 0.1, "dna_dA2": 1, "rna_rR2": 0, "Core2_OFF": 0.1, "dna_dA1": 1, "rna_rR1": 0.5, "protein_RNAseH": 1,
      "protein_RNAP":15})
# Create parameters
P.create_parameter("rna_rR2i", 10)
P.create_parameter("T_rna_rR2i", 1800)

# Create rule 1
P.create_rule("assignment", {"equation":"rna_rR2 = _rna_rR2i*Heaviside(t-_T_rna_rR2i)"})

#Simulate Model through bioscrape
timepoints = np.linspace (0, 28800, 1000)
results = py_simulate_model (timepoints, Model = P)
timepoints = timepoints/3600

# Plot Results


In [5]:
j = bokeh.plotting.figure(plot_width=400, plot_height=300, x_axis_label = x, y_axis_label = y)
j.circle(timepoints, results["rna_rR2"], legend_label = "rR2", color = "red")
j.legend.location = "bottom_right"

k = bokeh.plotting.figure(plot_width=400, plot_height=300, x_axis_label = x, y_axis_label = y)
k.circle(timepoints, results["complex_Core2_ON_protein_RNAP"], legend_label = "ON Core 2:RNAP", color = "red")
k.circle(timepoints, results["complex_Core1_ON_protein_RNAP"], legend_label = "ON Core 1:RNAP", color = "blue")
k.legend.click_policy="hide"
k.legend.location = "center_right"

#l = bokeh.plotting.figure(plot_width=400, plot_height=300, x_axis_label = x, y_axis_label = y)
#l.circle(timepoints, results["complex_Core2_OFF_protein_RNAP"], legend_label = "OFF Core 2:RNAP", color = "red")
#l.circle(timepoints, results["complex_Core1_OFF_protein_RNAP"], legend_label = "OFF Core 1:RNAP", color = "blue")
#l.legend.click_policy="hide"
#l.legend.location = "center_right"

m = bokeh.plotting.figure(plot_width=400, plot_height=300, x_axis_label = x, y_axis_label = y)
m.circle(timepoints, results["rna_rR1"], legend_label = "rR1", color = "blue")
m.legend.click_policy="hide"
m.legend.location = "center_right"

bokeh.io.show(row(k, l, j, m))
warnings.filterwarnings("ignore")

In [6]:
# Core 1 concentration/molecules
bokeh.io.output_notebook()
r = bokeh.plotting.figure(plot_width=400, plot_height=300, x_axis_label = x, y_axis_label = y)
r.circle(timepoints, results["Core1_OFF"], legend_label = "OFF Core 1", color = "blue")
r.circle(timepoints, results["complex_Core1_ON"], legend_label = "ON Core 1", color = "red")
r.legend.click_policy="hide"


# Core 2 concentration/molecules
s = bokeh.plotting.figure(plot_width=400, plot_height=300,  x_axis_label = x, y_axis_label = y)
s.circle(timepoints, results["Core2_OFF"], legend_label = "OFF Core 2", color = "darkblue")
s.circle(timepoints, results["complex_Core2_ON"], legend_label = "ON Core 2" , color = "darkred")
s.legend.click_policy="hide"

bokeh.io.show(row(r, s))
warnings.filterwarnings("ignore")

Loading BokehJS ...

In [7]:
n = bokeh.plotting.figure(plot_width=400, plot_height=300, x_axis_label = x, y_axis_label = y)
n.circle(timepoints, results["dna_dA1"], legend_label = "dA1", color = "orange")
n.legend.click_policy="hide"
n.legend.location = "top_right"

o = bokeh.plotting.figure(plot_width=400, plot_height=300, x_axis_label = x, y_axis_label = y)
o.circle(timepoints, results["dna_dA2"], legend_label = "dA2", color = "darkblue")
o.legend.click_policy="hide"
o.legend.location = "top_right"

bokeh.io.show(row(n, o))
warnings.filterwarnings("ignore")

In [8]:
p = bokeh.plotting.figure(plot_width=400, plot_height=300,  x_axis_label = x, y_axis_label = y)
p.circle(timepoints, results["complex_Core1_AI"], legend_label = "dA2:rR2", color = "darkblue")
p.legend.location="bottom_right"

q = bokeh.plotting.figure(plot_width=400, plot_height=300,  x_axis_label = x, y_axis_label = y)
q.circle(timepoints, results["complex_Core2_AI"], legend_label = "dA1:rR1", color = "darkred")

bokeh.io.show(row(p, q))

export_png(row(j, m, n, o), filename="BiSwitch_AddrR2_Species.png")

**Observations**
Changed to Schaffter and Schulman concentrations. Also, RNase H is not at proper concentration. Just set to 1.